Crear un py que ejecuta el codigo en este archivo en loop para todos los archivos de una carpeta

Este archivo realiza el filtrado de datos y creacion de las redes contextualzadas con los nodos experimento y sus respectivas relaciones, tambien se incorporan relaciones de Genes expresados, el ADR de hace se filtrando los genes expresados y que estan relacionados al nodo (TFs) para filtrarlos y contrastarlo con los TF expresados en otro/s experimentos).

In [15]:
import pandas as pd
import numpy as np 
from py2neo import Node,Relationship,Graph, database,NodeMatcher 
import sys

#file=sys.argv[1]

In [103]:
file1=pd.read_csv("abundance28.tsv",sep="\t")
file1.head(),file1.shape

(           target_id  length  eff_length  est_counts  tpm
 0  ENST00000632684.1      12     2.60424         0.0  0.0
 1  ENST00000434970.2       9     2.52391         0.0  0.0
 2  ENST00000448914.1      13     2.62202         0.0  0.0
 3  ENST00000415118.1       8     2.48058         0.0  0.0
 4  ENST00000631435.1      12     2.60424         0.0  0.0,
 (187626, 5))

Con el archivo como Dataframe se continua el mapeo del trancrito sobre el simbols 

In [17]:
simbols=pd.read_csv("t2g2s 4-10-21.txt",index_col=None,delimiter="\t")
simbols=simbols[['ensembl_transcript_id',"external_gene_name"]]
simbols.head(),len(simbols),type(simbols)

(  ensembl_transcript_id external_gene_name
 0       ENST00000387314              MT-TF
 1       ENST00000389680            MT-RNR1
 2       ENST00000387342              MT-TV
 3       ENST00000387347            MT-RNR2
 4       ENST00000386347             MT-TL1,
 256183,
 pandas.core.frame.DataFrame)

Sortear alfabeticamente los transcritos en t2g2s para poder hacer busqueda binaria

In [46]:
#simbols.sort_values(by="ensembl_transcript_id", inplace=True)

In [47]:
simbols.head(),len(simbols),type(simbols)

(       ensembl_transcript_id external_gene_name
 193441       ENST00000000233               ARF5
 108899       ENST00000000412               M6PR
 162936       ENST00000000442              ESRRA
 83564        ENST00000001008              FKBP4
 137415       ENST00000001146            CYP26B1,
 256183,
 pandas.core.frame.DataFrame)

In [49]:
simbols.reset_index(inplace=True)
simbols["ensembl_transcript_id"][0]

'ENST00000000233'

In [55]:
ENST=file1["target_id"]
ENST.head(),len(ENST)

(0    ENST00000632684.1
 1    ENST00000434970.2
 2    ENST00000448914.1
 3    ENST00000415118.1
 4    ENST00000631435.1
 Name: target_id, dtype: object,
 187626)

Busqueda binaria


In [37]:
def binary_search(sequence,item):
    begin_index=0
    end_index=len(sequence)-1
    while begin_index<=end_index:
        mid_point= begin_index +(end_index-begin_index)//2
        mid_point_value=sequence[mid_point]
        if mid_point_value==item:
            return (mid_point)

        elif item< mid_point_value:
            end_index=mid_point-1

        else:
            begin_index=mid_point+1
    return (None)

In [51]:
tar="ENST00000387314"
print(binary_search(simbols['ensembl_transcript_id'],tar))

29671


In [53]:
ENST_adj_simbol_list=[]
for i in range(0,len(ENST)):
    s=ENST[i]
    #print(s[:-2])
    res=binary_search(simbols['ensembl_transcript_id'],s[:-2])
    if res!=None:
        ENST_adj_simbol_list.append(simbols["external_gene_name"][res])
    else:
        ENST_adj_simbol_list.append(None)
ENST_adj_simbol_list[:5]

['TRBD1', 'TRDD2', 'TRDD3', 'TRDD1', 'TRBD1']

Check point: ver que los ensablajes y su respectivo simbol corresponden y estan ordenados.

In [78]:
q=len(ENST_adj_simbol_list)
s=ENST_adj_simbol_list.count(None)
p=round(s/q*100,3)
q,s,str(p)+"%"

(187626, 5637, '3.004%')

In [64]:
df_None= np.where(np.array(ENST_adj_simbol_list) == None)[0]
df_None[:15]

array([ 266,  267,  287,  359,  379,  924,  968, 1009, 1137, 1162, 1199,
       1239, 1281, 1294, 1354], dtype=int64)

In [70]:
a=ENST.loc[287]
if a[:-2] in simbols['ensembl_transcript_id']:
    print("Yes")
else:
    print("No")

No


Checkpint, no se encuentran el 3% de los transcritos de kallisto en la lista de ENST para esta version

indicar version index de kallisto:

indicar version ENST sacada desde R:

In [104]:
file1["Syb"]=ENST_adj_simbol_list
file1.head(), file1.shape

(           target_id  length  eff_length  est_counts  tpm    Syb
 0  ENST00000632684.1      12     2.60424         0.0  0.0  TRBD1
 1  ENST00000434970.2       9     2.52391         0.0  0.0  TRDD2
 2  ENST00000448914.1      13     2.62202         0.0  0.0  TRDD3
 3  ENST00000415118.1       8     2.48058         0.0  0.0  TRDD1
 4  ENST00000631435.1      12     2.60424         0.0  0.0  TRBD1,
 (187626, 6))

Agrupar los genes con el mismo simbol sumando los valores de conteo...


In [94]:
Conteo_gen=file1.groupby("Syb",as_index=False).sum()# simbols y el resto son columnas numericas

In [95]:
Conteo_gen.head(),Conteo_gen.shape

(       Syb  length   eff_length  est_counts         tpm
 0  5S_rRNA     867     36.79662     0.00000    0.000000
 1     A1BG    7549   6554.00000    55.14663   19.400249
 2     A1CF   46436  44446.00000     2.00000    0.034660
 3      A2M    6982   4594.00000   479.72632  241.094182
 4    A2ML1   10614   9022.00000     3.09675    0.270248,
 (36302, 5))

se crea al nodo experimento con sus respectivas propiedades, los nodos clasificadores de este experimento y sus relaciones

In [98]:
Conteo_gen[Conteo_gen["est_counts"]>0].shape,Conteo_gen[Conteo_gen["est_counts"]>10].shape

((19008, 5), (13260, 5))

In [100]:
graph=Graph()# or Graph('bolt://neo4j@localhost:7687')

In [132]:
file1_0=Conteo_gen[Conteo_gen["est_counts"]>10]
file1_0.head(),type(file1_0)

(       Syb  length  eff_length  est_counts         tpm
 1     A1BG    7549      6554.0    55.14663   19.400249
 3      A2M    6982      4594.0   479.72632  241.094182
 7   A4GALT    9985      8592.0   293.39200   30.360534
 9     AAAS   19227     15247.0   111.00006   12.079000
 10    AACS   19407     17417.0    18.54619    1.199477,
 pandas.core.frame.DataFrame)

In [134]:
file1_0.head(15)


,Syb,length,eff_length,est_counts,tpm
1,A1BG,7549,6554.0,55.146630,19.400249
3,A2M,6982,4594.0,479.726320,241.094182
7,A4GALT,9985,8592.0,293.392000,30.360534
9,AAAS,19227,15247.0,111.000060,12.079000
10,AACS,19407,17417.0,18.546190,1.199477
17,AADAT,11668,10076.0,17.000000,1.596668
18,AAGAB,9786,8393.0,369.000000,26.817700
19,AAK1,34335,31748.0,225.623440,7.801211
20,AAMDC,6197,4008.0,62.111506,27.504305
21,AAMP,13684,11694.0,671.000290,71.927232


In [133]:
 genes_in_db=[]
 for i in file1_0["Syb"]:
    b=graph.run("MATCH(b:Gene{name:$name1}) return b.name",name1=i).evaluate()
    if b!=i:
        genes_in_db.append(i)#en este caso incluye genes repetidos
genes_in_db[:10],len(genes_in_db)

(['ABCF2-H2BE1',
  'ABRAXAS2',
  'AC002996.1',
  'AC004057.1',
  'AC004696.1',
  'AC004832.3',
  'AC004890.2',
  'AC004922.1',
  'AC004980.1',
  'AC004980.5'],
 845)

In [ ]:
for i in range(0,len(Conteo_gen["est_counts"]):
    

In [ ]:

???graph.run("MERGE(a:Experiment{name:$code,condition:$condition,comentarios:$com})",code="GSM4462413"condition="Healthy", com="lo que se quiera agregar de los genes que no se incorporaron ")#nombre del experimento, tipo de muestra(control o sano)

In [ ]:
for i in range(0,5):
        #graph.run("MERGE(a:Experiment{code:$code,condition:$condition}) MATCH(b:Gene{name:$simb} CREATE (a)-[:RES{count:$count}]->(b) ",code="GSM4462413",condition="Healthy",simb=res["simb"][i], count=res["count"][i])
    else:
        print(i)

Se realiza la reacion de relaciones entre el nodo experimento con sus respectivos genes

In [ ]:
graph.run("MERGE(a:Experiment{name:$code})",code="GSM4462413").evaluate()

In [ ]:
graph.run("MERGE(a:Experiment{name:$code,condition:$condition,comentarios:$com})",code="GSM4462413"condition="Healthy", com="lo que se quiera agregar de los genes que no se incorporaron ")

In [ ]:
for i in range(0,len(res)):
    if res["healthy"][i]>10:
        print(res["Syb"][i])
        #graph.run("MERGE(a:EXPERIMENT{code:$code,condition:$condition}) MATCH(b:TF{name:$tf} CREATE (a)-[:DEG]->(b) ",code="GSM4462413",condition="Healthy", tf=res["Syb"][i])
    else:
        print(i)